In [8]:
import os
os.chdir("C:\Users\suh0902\Desktop\CP\dataset")
import pickle
import numpy
from time import time

from sklearn import cross_validation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectPercentile, f_classif


In [3]:
def preprocess(words_file = "word_data.pkl", authors_file="email_authors.pkl"):
    """ 
        this function takes a pre-made list of email texts (by default word_data.pkl)
        and the corresponding authors (by default email_authors.pkl) and performs
        a number of preprocessing steps:
            -- splits into training/testing sets (10% testing)
            -- vectorizes into tfidf matrix
            -- selects/keeps most helpful features
        after this, the feaures and labels are put into numpy arrays, which play nice with sklearn functions
        4 objects are returned:
            -- training/testing features
            -- training/testing labels
    """

    ### the words (features) and authors (labels), already largely preprocessed
    ### this preprocessing will be repeated in the text learning mini-project
    word_data = pickle.load(open(words_file, "r"))
    authors = pickle.load(open(authors_file, "r"))

    ### test_size is the percentage of events assigned to the test set (remainder go into training)
    features_train, features_test, labels_train, labels_test = cross_validation.train_test_split(word_data, authors, test_size=0.1, random_state=42)



    ### text vectorization--go from strings to lists of numbers
    vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
    features_train_transformed = vectorizer.fit_transform(features_train)
    features_test_transformed  = vectorizer.transform(features_test)



    ### feature selection, because text is super high dimensional and 
    ### can be really computationally chewy as a result
    selector = SelectPercentile(f_classif, percentile=10)
    selector.fit(features_train_transformed, labels_train)
    features_train_transformed = selector.transform(features_train_transformed).toarray()
    features_test_transformed  = selector.transform(features_test_transformed).toarray()

    ### info on the data
    print "no. of Chris training emails:", sum(labels_train)
    print "no. of Sara training emails:", len(labels_train)-sum(labels_train)


    return features_train_transformed, features_test_transformed, labels_train, labels_test

In [5]:
features_train, features_test, labels_train, labels_test = preprocess()

no. of Chris training emails: 7936
no. of Sara training emails: 7884


In [6]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [7]:
features_train = features_train[:len(features_train)/100]
labels_train = labels_train[:len(labels_train)/100]

In [9]:
clf = SVC(kernel="rbf",C= 1000)
t0=time()
clf.fit(features_train, labels_train)
print "Training time:", round(time()-t0,3), "s"

Training time: 0.236 s


In [10]:
t0=time()
pred = clf.predict(features_test)
print "Predicting time:", round(time()-t0,3), "s"

Predicting time: 1.277 s


In [11]:
print pred[10], pred[26], pred[50]
acc = accuracy_score(pred, labels_test)
print acc

1 0 1
0.821387940842


In [12]:
def sum_pred(n):
    count = 0
    for res in pred:
        if n==1:
           count +=1
    return count
print sum_pred(1)

1758
